In [18]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/{search} 뉴스원문{num}.csv')

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)



# 제목과 내용을 합쳐서 content 열 생성????
# 왜 그냥은 안되냐 title로 하면 왜 안되냐..???? 이거 수정하고 결국에는 라벨링이 필요하다......
df['content'] = df['title']

print(type([df['title']]))
# content 열에 대해 형태소 분석 적용
df['content'] = df['content'].apply(tokenize)

# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# content 열에 대해 예측 수행
df['sentiment'] = df['content'].apply(lambda x: predict(preprocess(' '.join(x))))

# 감성 분석 결과를 문자 형식으로 변환
# 조금 더 디테일하게 4.7 이하 부정, 5.3 이상 긍정, 그 외 중립 추가
def convert_sentiment(probs):
    if probs[0] <= 0.47:
        return '부정'
    elif probs[0]>= 0.53:
        return '긍정'
    else:
        return '중립'
imsi = df['sentiment']
df['sentiment'] = df['sentiment'].apply(convert_sentim

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 8


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'list'>


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
# csv 파일로 저장
df.to_csv(f'csv/{search}_주가뉴스(KoBart season2){num}.csv', index=False)

# 결과 확인
print(df['content'])
print(df['sentiment'])

PermissionError: [Errno 13] Permission denied: 'csv/삼성전자_주가뉴스(KoBart season2)8.csv'

In [20]:
df['sentiment']

0    긍정
1    중립
2    긍정
3    긍정
4    중립
5    부정
6    긍정
7    중립
8    긍정
Name: sentiment, dtype: object

In [21]:
imsi

0     [0.6182462, 0.38175377]
1     [0.5032322, 0.49676776]
2      [0.6365739, 0.3634261]
3    [0.61342347, 0.38657644]
4       [0.50127, 0.49873003]
5     [0.44513664, 0.5548634]
6     [0.53225046, 0.4677495]
7    [0.48493963, 0.51506037]
8      [0.5667302, 0.4332698]
Name: sentiment, dtype: object

In [22]:
total = 0
for i in range(11):
    total += imsi[i][0]
avg = total/10

KeyError: 9

In [23]:
type(imsi[1][0])

numpy.float32

In [24]:
# 갯수로 접근할거냐 평균으로 접근할거냐???????
type(df['sentiment'])

pandas.core.series.Series

In [25]:
imsi[1]

array([0.5032322 , 0.49676776], dtype=float32)